<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd 
import os 

import re 
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import f1_score
from  sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

In [6]:
data = pd.read_csv('toxic_comments.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'toxic_comments.csv'

Как мы можем заметить, в нашем Датафрейме отсутствуют пропуски, нет дубликатов а значит мы можем переходить к следующему этапу

Нам нужно создать столбец, в котором наши тексты будут очищены и лемматизированы.

In [ ]:
# Напишем функцию для очищения текста
def clear_text(text):
     return re.sub(r'[^A-Za-z ]', ' ', str(text))

In [ ]:
features_downsampled = data['text'].str.lower()

Токенизируем комментарии(разделим их на отдельные слова)

In [ ]:
# Напишем функцию для лемматизации 
def lemmatize(text):
    wnl = WordNetLemmatizer()
    lemm_list = wnl.lemmatize(text)
    lemm_text = ''.join(lemm_list)
    return lemm_text

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict={"J":wordnet.ADJ,
             "N": wordnet.NOUN,
             "V": wordnet.VERB,
             "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return(" ".join(lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in w_tokenizer.tokenize(text)))

In [ ]:
plt.title('Распределение целевого признака')
data['toxic'].hist();

Наблюдается явный дисбаланс классов.Значений "1" много меньше чем "0".Поэтому нам надо будет применить баланс классов.

In [ ]:
data['text'] = data['text'].str.lower()
data['lemm'] = data['text'].apply(lemmatize)
data['lemm'] = data['lemm'].apply(clear_text)
data['lemm']

Мы лемматизировали тексты, избавились от лишних знаков,привели к нижнему регистру.

In [ ]:
features_train,features_test,target_train,target_test = train_test_split(features_downsampled,data['toxic'],test_size=0.2,random_state=12345)
features_train,features_valid,target_train,target_valid = train_test_split(features_train,target_train,test_size=0.4,random_state=12345)

Разделили данные на обучающую,валидационную и тестовую выборки.

In [ ]:
stopwords = set(nltk_stopwords.words('english'))

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(features_train,y=None)
tf_idf_valid = count_tf_idf.transform(features_valid)
tf_idf_test = count_tf_idf.transform(features_test)
print(tf_idf_train.shape, tf_idf_valid.shape,tf_idf_test.shape)

Векторизируем наши тексты.

Мы преобразовали наши данные, а именно, привели к нижнему регистру, лемматизировали, очистили от стопслов.Проследили за распределением целевого признака.Векторизировали наши строки.Разделили данные на выборки.Теперь можем переходить к обучению моделей.

## Обучение

In [ ]:
model = LogisticRegression(class_weight='balanced',random_state=12345)
model.fit(tf_idf_train,target_train)
predictions = model.predict(tf_idf_valid)
print('F1_score: ',f1_score(target_valid,predictions))

Получили f1_score равный 0.74.

In [ ]:
%%time
grid = [{'max_depth':range(2,9)}]
tree = DecisionTreeClassifier(class_weight='balanced',random_state=12345)
model = GridSearchCV(tree,grid,verbose=1)
model.fit(tf_idf_train,target_train)
predictions=model.predict(tf_idf_valid)
print('F1-score: ', f1_score(target_valid,predictions))
print(model.best_params_)

In [ ]:
%%time
grid = [{'n_estimators' : [100],'max_depth':range(2,9)}]
forest = RandomForestClassifier(class_weight='balanced',random_state=12345)
model = GridSearchCV(forest,grid,verbose=1)
model.fit(tf_idf_train,target_train)
predictions=model.predict(tf_idf_valid)
print('F1-score: ', f1_score(target_valid,predictions))
print('Метрика равна ', model.best_score_)
print(model.best_params_)

Мы обучили несколько моеделей классификатора,применили баланс классов и нашли значения метрик для них \
Для Логистической Регрессии мы получили 0.74\
Для Дерева решений мы получили лучший результат при глубине 8.Метрика равна 0.52\
Для Случайного Леса метрика равна 0.68.\
Модель с наибольшей метрикой - Логистическая Регрессия

## Выводы

In [ ]:
model = LogisticRegression(class_weight='balanced',random_state=12345)
model.fit(tf_idf_train,target_train)
predictions = model.predict(tf_idf_test)
print(f1_score(target_test,predictions))

Мы получили F1_score больший 0.75.Значит модель хорошо предсказывает целевой класс.

# Вывод
Сначала мы подготовили наши данные.Провели их анализ. Заметили несбалансированность целевого признака.Мы преобразовали наши данные, а именно, привели к нижнему регистру, лемматизировали, очистили от стопслов.Проследили за распределением целевого признака.Векторизировали наши строки.Разделили данные на выборки.Обучили разные модели.Модель с наибольшей метрикой - Логистическая Регрессия.Мы получили F1_score больший 0.75.Значит наша метрика прошла проверку.Модель нам подходит.